In [ ]:
import os
import pandas as pd
from sklearn.metrics import r2_score

In [ ]:
growthCheckTargetList = ["totalSeedWeightAfterDry", "totalSeedNum", "totEarWeight", "totEarNum", "SPAD", "LAI", "Height", "20StrawWeightAfterDry"]
errorModel = ["ConvNeXtTinyWheatModelWithConfidence", "FocalNetConfidenceModel"]

def calculateAccuracyRegardingDays(predictionFilePath):
    ''' 
    grouping result by date
    and calculate each R2 and return the result as df
    '''
    # read the csv file
    df = pd.read_csv(predictionFilePath)

    # grouping all data by date from RGB path
    df["date"] = df["rgb_path"].apply(lambda x: x.split("/")[-1].split("_")[1][:8])
    dataGroupedByDate = df.groupby("date")

    # loop and calculate each group R2 and predicted confidence
    results = []
    for date, group_df in dataGroupedByDate:
        r2 = r2_score(group_df["true"], group_df["predicted"])
        avg_std = group_df["predicted_std"].mean()
        results.append((date, r2, avg_std))    

    results_df = pd.DataFrame(results, columns=["date", "r2_score", "avg_predicted_std"])
    return results_df

def loopCallCalculateAccuracyRegardingDays(predictionPath, saveFolderPath = "./growthPredictions/resultByDate/" ):
    ''' 
    loop to each prediccted csv
    get rid of the unused file 
    call calculate group R2 function
    save result to folder
    '''
    # create output folder
    os.makedirs(saveFolderPath, exist_ok=True)

    # loop into each csv file
    for predictionFile in os.listdir(predictionPath):
        # check if the file is nused file: start with ".", not target data, use error model
        if(predictionFile[0] == '.' or predictionFile.split("_")[0] not in growthCheckTargetList or predictionFile.split("_")[1] in errorModel):
            continue

        # create file path
        predictionFilePath = os.path.join(predictionPath, predictionFile)

        # call calculate group R2 
        # print(predictionFile)
        results_df = calculateAccuracyRegardingDays(predictionFilePath)

        savePath = saveFolderPath + predictionFile.split('.')[0] + "_date.csv"
        print(savePath)
        results_df.to_csv(savePath, index=False)



In [ ]:
originalPredictionFolderPath = "./growthPredictions/predictions"
loopCallCalculateAccuracyRegardingDays(originalPredictionFolderPath)